# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


In [ ]:
# !wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py
# !wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_diffusers_to_original_stable_diffusion.py

In [ ]:
!pip install -U -qq git+https://github.com/huggingface/diffusers.git
!pip install -qq accelerate tensorboard transformers ftfy gradio xformers
!pip install -qq "ipywidgets>=7,<8"
!pip install -qq bitsandbytes
!pip install -U --pre triton 
!pip install wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.2 M

## Imports

In [ ]:
import gdown
import numpy as np
import PIL
from PIL import Image

from os.path import exists

from tqdm.auto import tqdm
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

from functools import partialmethod

from os.path import exists, join

import io
import imageio

In [ ]:
DATA_DIR = '../datasets/generated_datasets'

if exists("/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion"):
  WORK_DIR = "/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion"
else:
  WORK_DIR = "/content/drive/MyDrive/ORT/Tesis/Codes/diffusion"

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

## HF token 

In [ ]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = ""
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v2-1-pneumoconiosis-finetuned-lora"

# Load the pipeline with the same arguments (model, revision) that were used for training
model_id = "stabilityai/stable-diffusion-2-1"
unet_path = OUTPUT_DIR + "/pytorch_lora_weights.bin"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, safety_checker=None, torch_dtype=torch.float16, feature_extractor=None, requires_safety_checker=False)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe.unet.load_attn_procs(unet_path)
pipe.to(DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_hf_folder.py:92: UserWarning: A token has been found in `/root/.huggingface/token`. This is the old path where tokens were stored. The new location is `/root/.cache/huggingface/token` which is configurable using `HF_HOME` environment variable. Your token has been copied to this new location. You can now safely delete the old token file manually or use `huggingface-cli logout`.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.17.0.dev0",
  "feature_extractor": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DPMSolverMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
def generate_images(num_imgs: int, prompt: str, silent: bool=False) -> np.array: 
    guidance_scale = 7.5 
    num_inference_steps = 24 
    height = 512 
    width = 512 
    
 
    if silent:
        original = tqdm.__init__
        tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)
    
    with torch.inference_mode():
        gen_imgs = pipe(
            prompt,
            height=height,
            width=width,
            num_images_per_prompt=num_imgs,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale
        ).images
    
    if silent:
        tqdm.__init__ = original
        
    return gen_imgs

In [ ]:
IMAGE_FOLDER = WORK_DIR + '/' + DATA_DIR + '/pneumoconiosis' + '/diffusion' + '/StableDiffusionV2' + '/dreambooth_V2-1_finetuned-lora'
if not exists(IMAGE_FOLDER):
    !mkdir "$IMAGE_FOLDER"
IMAGE_FOLDER

'/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/../datasets/generated_datasets/pneumoconiosis/diffusion/StableDiffusionV2/dreambooth_V2-1_finetuned-lora'

In [ ]:
num_images = 114
for i in tqdm(range(num_images), total=num_images):
    gen_image = generate_images(1, "image of a pneumoconiosis xray", silent=True)[0]
    # monochannel:
    gen_image = gen_image.convert('L')
    imageio.imwrite(join(IMAGE_FOLDER, f"{i}.png"), gen_image)

  0%|          | 0/114 [00:00<?, ?it/s]